# Deploy to Azure Kubernetes Service (AKS)

To deploy your MLflow model to an Azure Machine Learning web service, your model must be set up with the MLflow Tracking URI to connect with Azure Machine Learning.

To deploy to AKS, first create an AKS cluster. Create an AKS cluster using the ComputeTarget.create() method. It may take 20-25 minutes to create a new cluster.

## Prerequisites 

* A machine learning model. If you don't have a trained model, find the notebook example that best fits your compute scenario in this [repo](https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/ml-frameworks/using-mlflow) and follow its instructions.
* [Set up the MLflow Tracking URI to connect Azure Machine Learning](https://docs.microsoft.com/en-us/azure/machine-learning/v1/how-to-use-mlflow#track-local-runs).
* Install the azureml-mlflow package.
    * This package automatically brings in azureml-core of the [Azure Machine Learning Python SDK](https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install), which provides the connectivity for MLflow to access your workspace.
* See which access permissions you need to perform your MLflow operations with your workspace


In [5]:
import mlflow
import mlflow.sklearn
import azureml.core
from azureml.core import Workspace
import matplotlib.pyplot as plt

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

ws = Workspace.from_config()

SDK version: 1.39.0


In [6]:
from azureml.core.compute import ComputeTarget, AksCompute
from azureml.core.compute_target import ComputeTargetException

aks_name = "my-aks-insights"

creating_compute = False
try:
    aks_target = ComputeTarget(ws, aks_name)
    print("Using existing AKS compute target {}.".format(aks_name))
except ComputeTargetException:
    print("Creating a new AKS compute target {}.".format(aks_name))

    # Use the default configuration (can also provide parameters to customize).
    prov_config = AksCompute.provisioning_configuration()
    aks_target = ComputeTarget.create(workspace=ws,
                                      name=aks_name,
                                      provisioning_configuration=prov_config)
    creating_compute = True

Using existing AKS compute target my-aks-insights.


Create a deployment config json using deploy_configuration() method values as a reference. Each of the deployment config parameters simply need to be defined as a dictionary. Here's an example below:

In [1]:
%%writefile deployment_config.json
{"computeType": "aks", "computeTargetName": "my-aks-insights"}

Overwriting deployment_config.json


In [3]:
import mlflow
from mlflow.deployments import get_deploy_client

# set the tracking uri as the deployment client
client = get_deploy_client(mlflow.get_tracking_uri())

# set the model path 
model_path = "model"

# set the deployment config
deploy_path = "deployment_config.json"
test_config = {'deploy-config-file': deploy_path}

# define the model path and the name is the service name
# the model gets registered automatically and a name is autogenerated using the "name" parameter below 
client.create_deployment(model_uri='runs:/{}/{}'.format(run.id, model_path),
                         config=test_config,
                         name="mlflow-test-aks")

MlflowException: No plugin found for managing model deployments to "file". In order to deploy models to "file", find and install an appropriate plugin from https://mlflow.org/docs/latest/plugins.html#community-plugins using your package manager (pip, conda etc).